# Geocoding Script
This R script is used mainly to geocode each individual data point to obtain their respective longitude and latitude. This will enable us to visualise the geographic location of each resale flat sale on Tableau.

Please refer to the comments in the respective code block for more details and explanations. 

In [1]:
#Import and load required packages 
#Register the required Google API 

library(readr)
library(ggmap) 
library(dplyr)

register_google(key="")

Warning message:
"package 'readr' was built under R version 3.6.1"Warning message:
"package 'ggmap' was built under R version 3.6.1"Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.6.1"Google's Terms of Service: https://cloud.google.com/maps-platform/terms/.
Please cite ggmap if you use it! See citation("ggmap") for details.
Warning message:
"package 'dplyr' was built under R version 3.6.1"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [2]:
#Load data set and view first few rows 

resale <- read_csv("resale_clean.csv")
head(resale) 

Warning message:
"Missing column names filled in: 'X1' [1]"Parsed with column specification:
cols(
  X1 = col_double(),
  month = col_character(),
  town = col_character(),
  flat_type = col_character(),
  block = col_character(),
  street_name = col_character(),
  storey_range = col_character(),
  floor_area_sqm = col_double(),
  flat_model = col_character(),
  lease_commence_date = col_double(),
  remaining_lease = col_double(),
  resale_price = col_double(),
  year = col_double(),
  months = col_double()
)


X1,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,year,months
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44,Improved,1979,61.33,232000,61,4
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67,New Generation,1978,60.58,250000,60,7
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62.42,262000,62,5
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68,New Generation,1980,62.08,265000,62,1
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62.42,265000,62,5
5,2017-01,ANG MO KIO,3 ROOM,150,ANG MO KIO AVE 5,01 TO 03,68,New Generation,1981,63.00,275000,63,0


In [3]:
#Create a new data set containing only rows in Jurong West, Tampines and Woodlands.

resale_3towns <- resale %>% 
            subset(select = -X1) %>%
            mutate(address = paste(block, street_name))

resale_3towns <- filter(resale_3towns, town %in% c('JURONG WEST', 'TAMPINES','WOODLANDS'))


head(resale_3towns)

month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,year,months,address
2017-01,JURONG WEST,2 ROOM,990A,JURONG WEST ST 93,01 TO 03,56,Premium Apartment,2008,90.58,240000,90,7,990A JURONG WEST ST 93
2017-01,JURONG WEST,2 ROOM,990B,JURONG WEST ST 93,07 TO 09,52,Premium Apartment,2008,90.58,258000,90,7,990B JURONG WEST ST 93
2017-01,JURONG WEST,3 ROOM,185,BOON LAY AVE,19 TO 21,59,Improved,1975,57.75,240000,57,9,185 BOON LAY AVE
2017-01,JURONG WEST,3 ROOM,119,HO CHING RD,01 TO 03,67,Improved,1972,54.00,240000,54,0,119 HO CHING RD
2017-01,JURONG WEST,3 ROOM,174,BOON LAY DR,04 TO 06,59,Improved,1978,60.50,245000,60,6,174 BOON LAY DR
2017-01,JURONG WEST,3 ROOM,113,TAO CHING RD,07 TO 09,67,Improved,1971,53.42,245000,53,5,113 TAO CHING RD


In [4]:
glimpse(resale_3towns)

Observations: 13,165
Variables: 14
$ month               <chr> "2017-01", "2017-01", "2017-01", "2017-01", "20...
$ town                <chr> "JURONG WEST", "JURONG WEST", "JURONG WEST", "J...
$ flat_type           <chr> "2 ROOM", "2 ROOM", "3 ROOM", "3 ROOM", "3 ROOM...
$ block               <chr> "990A", "990B", "185", "119", "174", "113", "17...
$ street_name         <chr> "JURONG WEST ST 93", "JURONG WEST ST 93", "BOON...
$ storey_range        <chr> "01 TO 03", "07 TO 09", "19 TO 21", "01 TO 03",...
$ floor_area_sqm      <dbl> 56, 52, 59, 67, 59, 67, 59, 67, 73, 64, 73, 73,...
$ flat_model          <chr> "Premium Apartment", "Premium Apartment", "Impr...
$ lease_commence_date <dbl> 2008, 2008, 1975, 1972, 1978, 1971, 1978, 1972,...
$ remaining_lease     <dbl> 90.58, 90.58, 57.75, 54.00, 60.50, 53.42, 60.50...
$ resale_price        <dbl> 240000, 258000, 240000, 240000, 245000, 245000,...
$ year                <dbl> 90, 90, 57, 54, 60, 53, 60, 54, 70, 67, 71, 70,...
$ months         

In [5]:
#Create a function that takes in a human readable address and geocodes to obtain the exact longitude and latitude
#We will use the Google geocoding API to perform this function 

get_lonlat <- function(x) {
    geocode(x)
}

In [6]:
#lon_lat <- get_lonlat(resale_3towns$address)

In [7]:
#Merge the data set containing the longitude and latitude with the original data set 

resale_geocoded <- merge(resale_3towns, lon_lat, by="row.names", quiet = TRUE)
resale_geocoded <- subset(resale_geocoded, select = -Row.names)
head(resale_geocoded)

ERROR: Error in as.data.frame(y): object 'lon_lat' not found


In [ ]:
#Export the new geocoded data set 

write.csv(resale_geocoded, "resale_geocoded_3towns.csv")

## Data Cleaning and Vetting
This section is primarily to check for any errors in the data and clean it where necessary. We will manually code and assign the correct longitudes and latitudes to rows with incorrect longitude/latitude values. 

In [85]:
#Load the relevant packages

library(mdsr)
library(dplyr)
library(readr)

In [86]:
#Assign the geocoded data set to a new object 

resale_geo_edit <- read_csv("resale_geocoded_3towns.csv")
resale_geo_edit <- subset(resale_geo_edit, select = -X1)
head(resale_geo_edit)

Warning message:
"Missing column names filled in: 'X1' [1]"Parsed with column specification:
cols(
  X1 = col_double(),
  month = col_character(),
  town = col_character(),
  flat_type = col_character(),
  block = col_character(),
  street_name = col_character(),
  storey_range = col_character(),
  floor_area_sqm = col_double(),
  flat_model = col_character(),
  lease_commence_date = col_double(),
  remaining_lease = col_double(),
  resale_price = col_double(),
  year = col_double(),
  months = col_double(),
  address = col_character(),
  lon = col_double(),
  lat = col_double()
)


month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,year,months,address,lon,lat
2017-01,JURONG WEST,2 ROOM,990A,JURONG WEST ST 93,01 TO 03,56,Premium Apartment,2008,90.58,240000,90,7,990A JURONG WEST ST 93,103.6951,1.335340
2017-01,JURONG WEST,3 ROOM,421,JURONG WEST ST 42,07 TO 09,64,Simplified,1985,67.42,269000,67,5,421 JURONG WEST ST 42,103.7272,1.350266
2017-01,JURONG WEST,EXECUTIVE,274A,JURONG WEST AVE 3,04 TO 06,126,Apartment,2001,83.92,438000,83,11,274A JURONG WEST AVE 3,103.7036,1.352374
2017-04,JURONG WEST,5 ROOM,517C,JURONG WEST ST 52,04 TO 06,115,Improved,2000,82.17,591000,82,2,517C JURONG WEST ST 52,103.7201,1.345007
2019-03,WOODLANDS,5 ROOM,625B,WOODLANDS DR 52,10 TO 12,122,Improved,1998,78.58,435000,78,7,625B WOODLANDS DR 52,103.8000,1.433428
2019-03,WOODLANDS,5 ROOM,555,WOODLANDS DR 53,07 TO 09,115,Improved,1999,79.83,380000,79,10,555 WOODLANDS DR 53,103.7957,1.432327


In [87]:
summary(resale_geo_edit)

    month               town            flat_type            block          
 Length:13165       Length:13165       Length:13165       Length:13165      
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            
                                                                            
                                                                            
 street_name        storey_range       floor_area_sqm   flat_model       
 Length:13165       Length:13165       Min.   : 37.0   Length:13165      
 Class :character   Class :character   1st Qu.: 91.0   Class :character  
 Mode  :character   Mode  :character   Median :104.0   Mode  :character  
                                       Mean   :104.7                     
              

In [97]:
#Filter out rows with strange longitude/latitude values 

wrong_lonlat <- filter(resale_geo_edit, lat > 2 | lon < 100)

In [98]:
head(wrong_lonlat)

month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,year,months,address,lon,lat
2019-04,WOODLANDS,5 ROOM,748,WOODLANDS CIRCLE,01 TO 03,122,Improved,1998,77.83,363888,77,10,748 WOODLANDS CIRCLE,-77.33863,43.12836
2019-04,WOODLANDS,5 ROOM,747,WOODLANDS CIRCLE,07 TO 09,122,Improved,1998,77.92,392500,77,11,747 WOODLANDS CIRCLE,-77.33863,43.12836
2019-05,WOODLANDS,4 ROOM,728,WOODLANDS CIRCLE,04 TO 06,91,Model A2,1997,77.42,310000,77,5,728 WOODLANDS CIRCLE,-77.33863,43.12836
2019-05,WOODLANDS,4 ROOM,742,WOODLANDS CIRCLE,10 TO 12,103,Model A,1997,77.25,313000,77,3,742 WOODLANDS CIRCLE,-77.33863,43.12836
2019-05,WOODLANDS,4 ROOM,742,WOODLANDS CIRCLE,04 TO 06,100,Model A,1997,77.25,330000,77,3,742 WOODLANDS CIRCLE,-77.33863,43.12836
2019-06,WOODLANDS,4 ROOM,742,WOODLANDS CIRCLE,01 TO 03,102,Model A,1997,77.17,294000,77,2,742 WOODLANDS CIRCLE,-77.33863,43.12836


In [99]:
#Group the wrong_lonlat dataset by address and count the number of entries per address 

wrong_lonlat %>% group_by(address) %>% summarize(N=n())

address,N
721 WOODLANDS CIRCLE,6
728 WOODLANDS CIRCLE,15
739 WOODLANDS CIRCLE,11
742 WOODLANDS CIRCLE,11
747 WOODLANDS CIRCLE,4
748 WOODLANDS CIRCLE,3


We obtain the correct longitude and latitude values from Google Maps and map them manually to each address. 
(There are a few addresses not reflected in the above output as there was some error mapping them previously before it as discovered and rectified)

721 Woodlands Circle - lat: 1.444043, lon: 103.800338  
728 Woodlands Circle - lat: 1.442956, lon: 103.798970  
738 Woodlands Circle - lat: 1.444187, lon: 103.797592  
739 Woodlands Circle - lat: 1.444075, lon: 103.796771  
742 Woodlands Circle - lat: 1.443932, lon: 103.795862  
747 Woodlands Circle - lat: 1.443205, lon: 103.795067  
748 Woodlands Circle - lat: 1.445343, lon: 103.795202  
751 Woodlands Circle - lat: 1.445139, lon: 103.794319  

In [105]:
#Create a function that takes in a dataset (x)
#Checks the exact address, and replaces the longitude/latitude values if there is a match

clean_address <- function(x) {
    x %>% 
    mutate(lon=ifelse(address == '721 WOODLANDS CIRCLE', 103.800338, lon),
           lat=ifelse(address == '721 WOODLANDS CIRCLE', 1.444043, lat)) %>%
    mutate(lon=ifelse(address == '728 WOODLANDS CIRCLE', 103.798970, lon),
           lat=ifelse(address == '728 WOODLANDS CIRCLE', 1.442956, lat)) %>%
    mutate(lon=ifelse(address == '738 WOODLANDS CIRCLE', 103.797592, lon),
           lat=ifelse(address == '738 WOODLANDS CIRCLE', 1.444187, lat)) %>%
    mutate(lon=ifelse(address == '739 WOODLANDS CIRCLE', 103.796771, lon),
           lat=ifelse(address == '739 WOODLANDS CIRCLE', 1.444075, lat)) %>%
    mutate(lon=ifelse(address == '742 WOODLANDS CIRCLE', 103.795862, lon),
           lat=ifelse(address == '742 WOODLANDS CIRCLE', 1.443932, lat)) %>%
    mutate(lon=ifelse(address == '747 WOODLANDS CIRCLE', 103.795067, lon),
           lat=ifelse(address == '747 WOODLANDS CIRCLE', 1.443205, lat)) %>%
    mutate(lon=ifelse(address == '748 WOODLANDS CIRCLE', 103.795202, lon),
           lat=ifelse(address == '748 WOODLANDS CIRCLE', 1.445343, lat)) %>%
    mutate(lon=ifelse(address == '751 WOODLANDS CIRCLE', 103.794319, lon),
           lat=ifelse(address == '751 WOODLANDS CIRCLE', 1.445139, lat))
}

In [101]:
#Apply the function above to the original dataset and map it to a new object called 'resale_clean'

resale_clean <- resale_geo_edit %>%
do(clean_address(.))

In [106]:
#Check if there are any more rows with strange longitude/latitude values 

wrong_lonlat <- filter(resale_clean, lon < 100 | lat > 2)

In [108]:
#There shouldn't be any rows with the wrong longitude and latitude values 

wrong_lat %>% group_by(address) %>% summarize(N=n())

address,N


In [109]:
#Export the clean data set for dashboarding and data visualisation

write.csv(resale_clean, "resale_geocoded_3towns.csv")